In [10]:
pytesseract??

Type:        module
String form: <module 'pytesseract' from 'c:\\Users\\91978\\anaconda3\\lib\\site-packages\\pytesseract\\__init__.py'>
File:        c:\users\91978\anaconda3\lib\site-packages\pytesseract\__init__.py
Source:     
# flake8: noqa: F401
from .pytesseract import ALTONotSupported
from .pytesseract import get_languages
from .pytesseract import get_tesseract_version
from .pytesseract import image_to_alto_xml
from .pytesseract import image_to_boxes
from .pytesseract import image_to_data
from .pytesseract import image_to_osd
from .pytesseract import image_to_pdf_or_hocr
from .pytesseract import image_to_string
from .pytesseract import Output
from .pytesseract import run_and_get_multiple_output
from .pytesseract import run_and_get_output
from .pytesseract import TesseractError
from .pytesseract import TesseractNotFoundError
from .pytesseract import TSVNotSupported


__version__ = '0.3.13'

In [29]:
import cv2
import pytesseract
import numpy as np

# Configure Tesseract executable path (update it based on your system)
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files (x86)\Tesseract-OCR\tesseract.exe'

# Function to preprocess the image
def preprocess_image(image_path):
    image = cv2.imread(image_path)
    grayscale = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    _, binary = cv2.threshold(grayscale, 150, 255, cv2.THRESH_BINARY_INV)
    return image, grayscale, binary

# Function to extract text using Tesseract OCR
def extract_text(grayscale):
    custom_config = r'--oem 3 --psm 6'
    text = pytesseract.image_to_string(grayscale, config=custom_config)
    return text

# Function to detect and extract non-text elements (e.g., signatures, seals)
def extract_non_text_elements(binary, original_image):
    contours, _ = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    signature_contours = []
    seal_contours = []

    for contour in contours:
        x, y, w, h = cv2.boundingRect(contour)
        aspect_ratio = w / float(h)

        # Heuristic to classify signature and seal
        if 2.5 < aspect_ratio < 8.0 and w > 50 and h < 50:  # Likely a signature
            signature_contours.append((x, y, w, h))
        elif 0.8 < aspect_ratio < 1.2 and 50 < w < 200:  # Likely a seal
            seal_contours.append((x, y, w, h))

    # Extract and save detected elements with increased size
    padding = 10  # Increase the size by adding padding
    signatures = [original_image[max(0, y-padding):y+h+padding, max(0, x-padding):x+w+padding] for x, y, w, h in signature_contours]
    seals = [original_image[max(0, y-padding):y+h+padding, max(0, x-padding):x+w+padding] for x, y, w, h in seal_contours]

    return signatures, seals

# Function to save extracted elements
def save_elements(elements, prefix):
    for i, element in enumerate(elements):
        filename = f"{prefix}_{i+1}.png"
        cv2.imwrite(filename, element)
        print(f"Saved {prefix}: {filename}")

# Main function to process the image
def process_image(image_path):
    image, grayscale, binary = preprocess_image(image_path)

    # Extract and print text
    text = extract_text(grayscale)
    print("Extracted Text:\n", text)

    # Extract signatures and seals
    signatures, seals = extract_non_text_elements(binary, image)

    # Save results
    save_elements(signatures, "signature")
    save_elements(seals, "seal")

# Input image path
image_path = "New folder/test6.png"
process_image(image_path)


Extracted Text:
  

 




In [27]:
import cv2
import pytesseract
import numpy as np

# Configure Tesseract executable path (update it based on your system)
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files (x86)\Tesseract-OCR\tesseract.exe'

# Function to preprocess the image
def preprocess_image(image_path):
    image = cv2.imread(image_path)
    grayscale = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    _, binary = cv2.threshold(grayscale, 150, 255, cv2.THRESH_BINARY_INV)
    return image, grayscale, binary

# Function to remove text from the image
def remove_text(grayscale, original_image):
    custom_config = r'--oem 3 --psm 6'
    h, w = grayscale.shape

    # Extract bounding boxes of text detected by Tesseract
    boxes = pytesseract.image_to_boxes(grayscale, config=custom_config)
    for box in boxes.splitlines():
        b = box.split()
        x, y, x2, y2 = int(b[1]), int(b[2]), int(b[3]), int(b[4])
        x, y, x2, y2 = max(0, x), h - max(0, y2), min(w, x2), h - min(h, y)
        cv2.rectangle(original_image, (x, y), (x2, y2), (255, 255, 255), -1)

    return original_image

# Function to detect and extract non-text elements (e.g., signatures, seals)
def extract_non_text_elements(binary, original_image):
    contours, _ = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    non_text_mask = np.zeros_like(original_image, dtype=np.uint8)

    for contour in contours:
        x, y, w, h = cv2.boundingRect(contour)
        aspect_ratio = w / float(h)

        # Heuristic to classify signature and seal
        if (2.5 < aspect_ratio < 8.0 and w > 50 and h < 50) or (0.8 < aspect_ratio < 1.2 and 50 < w < 200):
            # Likely a signature or seal
            cv2.drawContours(non_text_mask, [contour], -1, (255, 255, 255), thickness=cv2.FILLED)

    return non_text_mask

# Function to overlay non-text elements on a blank canvas
def overlay_non_text_elements(original_image, non_text_mask):
    result = np.full_like(original_image, 255)  # Create a white canvas
    result[non_text_mask == 255] = original_image[non_text_mask == 255]
    return result

# Main function to process the image
def process_image(image_path):
    image, grayscale, binary = preprocess_image(image_path)

    # Remove text from the image
    image_without_text = remove_text(grayscale, image.copy())

    # Extract non-text elements (signatures and seals)
    non_text_mask = extract_non_text_elements(binary, image)

    # Combine the signatures and seals with the blank canvas
    final_image = overlay_non_text_elements(image_without_text, non_text_mask)

    # Save the final image
    cv2.imwrite("cleaned_image_with_signatures_seals.png", final_image)
    print("Saved cleaned image with signatures and seals: cleaned_image_with_signatures_seals.png")

# Input image path
image_path = "New folder/test5.png"
process_image(image_path)


Saved cleaned image with signatures and seals: cleaned_image_with_signatures_seals.png


In [24]:
import cv2
import pytesseract
import numpy as np

# Configure Tesseract executable path (update it based on your system)
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files (x86)\Tesseract-OCR\tesseract.exe'

# Function to preprocess the image
def preprocess_image(image_path):
    image = cv2.imread(image_path)
    grayscale = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    _, binary = cv2.threshold(grayscale, 150, 255, cv2.THRESH_BINARY_INV)
    return image, grayscale, binary

# Function to remove text from the image
def remove_text(grayscale, original_image):
    custom_config = r'--oem 3 --psm 6'
    h, w = grayscale.shape

    # Extract bounding boxes of text detected by Tesseract
    boxes = pytesseract.image_to_boxes(grayscale, config=custom_config)
    for box in boxes.splitlines():
        b = box.split()
        x, y, x2, y2 = int(b[1]), int(b[2]), int(b[3]), int(b[4])
        x, y, x2, y2 = max(0, x), h - max(0, y2), min(w, x2), h - min(h, y)
        cv2.rectangle(original_image, (x, y), (x2, y2), (255, 255, 255), -1)

    return original_image

# Function to detect and extract non-text elements (e.g., signatures, seals)
def extract_non_text_elements(binary, original_image):
    contours, _ = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    non_text_mask = np.zeros_like(original_image, dtype=np.uint8)

    for contour in contours:
        x, y, w, h = cv2.boundingRect(contour)
        aspect_ratio = w / float(h)

        # Heuristic to classify signature and seal
        if (2.5 < aspect_ratio < 8.0 and w > 50 and h < 50) or (0.8 < aspect_ratio < 1.2 and 50 < w < 200):
            # Likely a signature or seal
            cv2.drawContours(non_text_mask, [contour], -1, (255, 255, 255), thickness=cv2.FILLED)

    return non_text_mask

# Function to overlay non-text elements on a blank canvas
def overlay_non_text_elements(original_image, non_text_mask):
    result = np.full_like(original_image, 255)  # Create a white canvas
    result[non_text_mask == 255] = original_image[non_text_mask == 255]
    return result

# Main function to process the image
def process_image(image_path):
    image, grayscale, binary = preprocess_image(image_path)

    # Remove text from the image
    image_without_text = remove_text(grayscale, image.copy())

    # Extract non-text elements (signatures and seals)
    non_text_mask = extract_non_text_elements(binary, image)

    # Combine the signatures and seals with the blank canvas
    final_image = overlay_non_text_elements(image_without_text, non_text_mask)

    # Save the final image
    cv2.imwrite("cleaned_image_with_signatures_seals.png", final_image)
    print("Saved cleaned image with signatures and seals: cleaned_image_with_signatures_seals.png")

# Input image path
image_path = "New folder/test5.png"
process_image(image_path)


Saved cleaned image with signatures and seals: cleaned_image_with_signatures_seals.png


In [34]:
class SignatureAndSeal:
    def __init__(self, image_path):
        self.image_path = image_path
        self.image = cv2.imread(image_path)
        self.grayscale = cv2.cvtColor(self.image, cv2.COLOR_BGR2GRAY)
        self.binary = cv2.threshold(self.grayscale, 150, 255, cv2.THRESH_BINARY_INV)[1]

    def extract_text(self):
        custom_config = r'--oem 3 --psm 6'
        self.text = pytesseract.image_to_string(self.grayscale, config=custom_config)
        return self.text

    def extract_non_text_elements(self):
        contours, _ = cv2.findContours(self.binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        self.signature_contours = []
        self.seal_contours = []

        for contour in contours:
            x, y, w, h = cv2.boundingRect(contour)
            aspect_ratio = w / float(h)

            # Heuristic to classify signature and seal
            if 2.5 < aspect_ratio < 8.0 and w > 50 and h < 50:  # Likely a signature
                self.signature_contours.append((x, y, w, h))
            elif 0.8 < aspect_ratio < 1.2 and 50 < w < 200:  # Likely a seal
                self.seal_contours.append((x, y, w, h))

        # Extract and save detected elements with increased size
        padding = 10  # Increase the size by adding padding
        self.signatures = [self.image[max(0, y-padding):y+h+padding, max(0, x-padding):x+w+padding] for x, y, w, h in self.signature_contours]
        self.seals = [self.image[max(0, y-padding):y+h+padding, max(0, x-padding):x+w+padding] for x, y, w, h in self.seal_contours]

    def save_elements(self):
        for i, signature in enumerate(self.signatures):
            filename = f"signature_{i+1}.png"
            cv2.imwrite(filename, signature)
            print(f"Saved signature: {filename}")

        for i, seal in enumerate(self.seals):
            filename = f"seal_{i+1}.png"
            cv2.imwrite(filename, seal)
            
            print(f"Saved seal: {filename}")
            
            
# Input image path
image_path = "New folder/test6.png"
signature_and_seal = SignatureAndSeal(image_path)
signature_and_seal.extract_text()
signature_and_seal.extract_non_text_elements()
signature_and_seal.save_elements()


In [48]:
import cv2
import numpy as np
from pathlib import Path

def preprocess_image(image_path):
    """
    Preprocess the image for better signature and seal detection.
    """
    # Read the image
    img = cv2.imread(str(image_path))
    if img is None:
        raise ValueError("Could not read the image")
    
    # Convert to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    # Apply adaptive thresholding
    thresh = cv2.adaptiveThreshold(
        gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, 
        cv2.THRESH_BINARY_INV, 11, 2
    )
    
    return img, thresh

def detect_signature_seal(thresh_img):
    """
    Detect potential signature and seal regions using contour analysis.
    """
    # Find contours
    contours, _ = cv2.findContours(
        thresh_img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE
    )
    
    # Filter contours based on area and aspect ratio
    signature_seal_regions = []
    for contour in contours:
        area = cv2.contourArea(contour)
        x, y, w, h = cv2.boundingRect(contour)
        aspect_ratio = w / h if h != 0 else 0
        
        # Adjust these thresholds based on your specific needs
        if area > 500 and 0.2 < aspect_ratio < 5:
            signature_seal_regions.append((x, y, w, h))
    
    return signature_seal_regions

def extract_regions(img, regions, padding=10):
    """
    Extract the identified regions with padding and create a mask.
    """
    mask = np.zeros_like(img)
    
    for x, y, w, h in regions:
        # Add padding to the region
        x1 = max(0, x - padding)
        y1 = max(0, y - padding)
        x2 = min(img.shape[1], x + w + padding)
        y2 = min(img.shape[0], y + h + padding)
        
        # Add region to mask
        mask[y1:y2, x1:x2] = img[y1:y2, x1:x2]
    
    return mask

def main(input_path, output_path):
    """
    Main function to process the image and save the result.
    """
    # Convert paths to Path objects
    input_path = Path(input_path)
    output_path = Path(output_path)
    
    try:
        # Process image
        original_img, thresh_img = preprocess_image(input_path)
        
        # Detect signature and seal regions
        regions = detect_signature_seal(thresh_img)
        
        # Extract regions
        result = extract_regions(original_img, regions)
        
        # Save result
        cv2.imwrite(str(output_path), result)
        print(f"Successfully saved extracted signatures and seals to {output_path}")
        
    except Exception as e:
        print(f"Error processing image: {str(e)}")

# Example usage
if __name__ == "__main__":
    input_path = "New folder/test.png"  # Replace with your input image path
    output_path = f"output/extracted_signatures_seals{input_path[11:]}.jpg"  # Replace with desired output path
    main(input_path, output_path)

Successfully saved extracted signatures and seals to output\extracted_signatures_sealstest.png.jpg
